In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = (SparkSession
         .builder
         .appName("Airport performance delays")
         .getOrCreate())

23/05/13 22:27:04 WARN Utils: Your hostname, wedivv-H110M-S2V resolves to a loopback address: 127.0.1.1; using 192.168.1.44 instead (on interface wlp5s0)
23/05/13 22:27:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/13 22:27:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/13 22:27:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
csvFile = ("./data/7-departuredelays.csv")

In [4]:
schema = "date STRING, delay INT, distance INT, origin STRING, destination STRING"

In [5]:
df = (spark.read
    .schema(schema)
    .format("csv")
    .option("inferSchema", "true")
    .option("header", "true")
    .load(csvFile))

In [6]:
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)



In [7]:
df.createOrReplaceTempView("us_delay_flights_tbl")

flights whose distance is greater than 1,000 miles

In [8]:
spark.sql("""
    SELECT distance, origin, destination, delay
    FROM us_delay_flights_tbl WHERE distance > 1000
    ORDER BY distance DESC, delay DESC
""").show(10)

+--------+------+-----------+-----+
|distance|origin|destination|delay|
+--------+------+-----------+-----+
|    4330|   HNL|        JFK|  932|
|    4330|   JFK|        HNL|  922|
|    4330|   JFK|        HNL|  784|
|    4330|   JFK|        HNL|  175|
|    4330|   JFK|        HNL|  134|
|    4330|   JFK|        HNL|  123|
|    4330|   JFK|        HNL|  118|
|    4330|   HNL|        JFK|  115|
|    4330|   JFK|        HNL|  111|
|    4330|   HNL|        JFK|  110|
+--------+------+-----------+-----+
only showing top 10 rows



In [9]:
from pyspark.sql.functions import col, desc
(   
  df
    .select("distance", "origin", "destination", "delay")
    .where(col("distance") > 1000)
    .orderBy(desc("distance"), desc("delay"))
).show(10)

+--------+------+-----------+-----+
|distance|origin|destination|delay|
+--------+------+-----------+-----+
|    4330|   HNL|        JFK|  932|
|    4330|   JFK|        HNL|  922|
|    4330|   JFK|        HNL|  784|
|    4330|   JFK|        HNL|  175|
|    4330|   JFK|        HNL|  134|
|    4330|   JFK|        HNL|  123|
|    4330|   JFK|        HNL|  118|
|    4330|   HNL|        JFK|  115|
|    4330|   JFK|        HNL|  111|
|    4330|   HNL|        JFK|  110|
+--------+------+-----------+-----+
only showing top 10 rows



flights between San Francisco (SFO) and Chicago
(ORD) with at least a two-hour delay

In [10]:
spark.sql("""
    SELECT date, origin, destination, delay
    FROM us_delay_flights_tbl
    WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD'
    ORDER BY delay DESC
""").show(10)

+--------+------+-----------+-----+
|    date|origin|destination|delay|
+--------+------+-----------+-----+
|02190925|   SFO|        ORD| 1638|
|01031755|   SFO|        ORD|  396|
|01022330|   SFO|        ORD|  326|
|01051205|   SFO|        ORD|  320|
|01190925|   SFO|        ORD|  297|
|02171115|   SFO|        ORD|  296|
|01071040|   SFO|        ORD|  279|
|01051550|   SFO|        ORD|  274|
|03120730|   SFO|        ORD|  266|
|01261104|   SFO|        ORD|  258|
+--------+------+-----------+-----+
only showing top 10 rows



In [11]:
(
    df
    .select("date", "origin", "destination", "delay")
    .where((col("delay") > 120) & (col("origin") == 'SFO') & (col("destination") == 'ORD'))
    .orderBy(desc("delay"))
).show(5)


+--------+------+-----------+-----+
|    date|origin|destination|delay|
+--------+------+-----------+-----+
|02190925|   SFO|        ORD| 1638|
|01031755|   SFO|        ORD|  396|
|01022330|   SFO|        ORD|  326|
|01051205|   SFO|        ORD|  320|
|01190925|   SFO|        ORD|  297|
+--------+------+-----------+-----+
only showing top 5 rows



convert the date column into a readable format

In [12]:
spark.sql("""
SELECT date_format(from_unixtime(unix_timestamp(date, 'MMddHHmm')), 'MM-dd HH:mm') as date
FROM us_delay_flights_tbl
""").show(3)


+-----------+
|       date|
+-----------+
|01-01 12:45|
|01-02 06:00|
|01-02 12:45|
+-----------+
only showing top 3 rows



In [13]:
from pyspark.sql.functions import date_format, from_unixtime, unix_timestamp

df_updated = (
    df.select(
        date_format(
            from_unixtime(
            unix_timestamp('date', 'MMddHHmm')), 'MM-dd HH:mm').alias('date'), 
    *df.columns).drop(df.date) # organizing columns like a pro in: ../others/2-Updating And OrganizingColumns.ipynb
) 
df_updated.show(3)

+-----------+-----+--------+------+-----------+
|       date|delay|distance|origin|destination|
+-----------+-----+--------+------+-----------+
|01-01 12:45|    6|     602|   ABE|        ATL|
|01-02 06:00|   -8|     369|   ABE|        DTW|
|01-02 12:45|   -2|     602|   ABE|        ATL|
+-----------+-----+--------+------+-----------+
only showing top 3 rows



In [14]:
df_updated.createOrReplaceTempView("us_delay_flights_tbl")


In [15]:
spark.sql("""
SELECT
  date,  delay,  origin,  destination,
  CASE
    WHEN SUBSTR(date, 1, 2) IN ('12', '01', '02') THEN 'Winter'
    ELSE 'Not Winter'
  END AS winter,
  CASE
    WHEN SUBSTR(date, 1, 2) = '01' AND SUBSTR(date, 4, 2) = '01' THEN 'New Years Day'
    WHEN SUBSTR(date, 1, 2) = '07' AND SUBSTR(date, 4, 2) = '04' THEN 'Independence Day'
    WHEN SUBSTR(date, 1, 2) = '12' AND SUBSTR(date, 4, 2) = '25' THEN 'Christmas Day'
    ELSE NULL
  END AS holiday
FROM us_delay_flights_tbl
WHERE delay > 120 AND origin = 'SFO' AND destination = 'ORD'
ORDER BY date ASC
""").show(5)

+-----------+-----+------+-----------+------+-------------+
|       date|delay|origin|destination|winter|      holiday|
+-----------+-----+------+-----------+------+-------------+
|01-01 12:37|  122|   SFO|        ORD|Winter|New Years Day|
|01-01 14:10|  124|   SFO|        ORD|Winter|New Years Day|
|01-02 07:20|  145|   SFO|        ORD|Winter|         null|
|01-02 12:05|  154|   SFO|        ORD|Winter|         null|
|01-02 14:10|  190|   SFO|        ORD|Winter|         null|
+-----------+-----+------+-----------+------+-------------+
only showing top 5 rows



Were the delays related to
winter months or holidays?

In [16]:
spark.sql("""
    SELECT winter, holiday, COUNT(*) as num_flights, ROUND(AVG(delay),2) as average_delay
    FROM (
SELECT
  date,  delay,  origin,  destination,
  CASE
    WHEN SUBSTR(date, 1, 2) IN ('12', '01', '02') THEN 'Winter'
    ELSE 'Not Winter'
  END AS winter,
  CASE
    WHEN SUBSTR(date, 1, 2) = '01' AND SUBSTR(date, 4, 2) = '01' THEN 'New Years Day'
    WHEN SUBSTR(date, 1, 2) = '07' AND SUBSTR(date, 4, 2) = '04' THEN 'Independence Day'
    WHEN SUBSTR(date, 1, 2) = '12' AND SUBSTR(date, 4, 2) = '25' THEN 'Christmas Day'
    ELSE NULL
  END AS holiday
FROM us_delay_flights_tbl
WHERE delay > 120 AND origin = 'SFO' AND destination = 'ORD'
ORDER BY date ASC
    )
    GROUP BY holiday, winter
""").show(5)

# Look like when it is winter the average delay tends to double.
# Not enough data flights for holidays (and the data only covers from january to march)


+----------+-------------+-----------+-------------+
|    winter|      holiday|num_flights|average_delay|
+----------+-------------+-----------+-------------+
|    Winter|         null|         37|       234.19|
|    Winter|New Years Day|          2|        123.0|
|Not Winter|         null|         16|       168.69|
+----------+-------------+-----------+-------------+



In [17]:
spark.sql("""
    SELECT delay, origin, destination,
        CASE
            WHEN delay > 360 THEN 'Very Long Delays'
            WHEN delay > 120 AND delay < 360 THEN 'Long Delays'
            WHEN delay > 60 AND delay < 120 THEN 'Short Delays'
            WHEN delay > 0 and delay < 60 THEN 'Tolerable Delays'
            WHEN delay = 0 THEN 'No Delays'
        ELSE 'Early'
        END AS Flight_Delays
    FROM us_delay_flights_tbl
    ORDER BY origin, delay DESC
""").show(10)

+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  Long Delays|
|  305|   ABE|        ATL|  Long Delays|
|  275|   ABE|        ATL|  Long Delays|
|  257|   ABE|        ATL|  Long Delays|
|  247|   ABE|        ATL|  Long Delays|
|  247|   ABE|        DTW|  Long Delays|
|  219|   ABE|        ORD|  Long Delays|
|  211|   ABE|        ATL|  Long Delays|
|  197|   ABE|        DTW|  Long Delays|
|  192|   ABE|        ORD|  Long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows



In [18]:
from pyspark.sql.functions import when

(
    df_updated
    .select(
        "delay", "origin", "destination",
        when(col("delay") > 360, "Very Long Delays")
        .when(col("delay") > 120, "Long Delays")
        .when(col("delay") > 60, "Short Delays")
        .when(col("delay") > 0, "Tolerable Delays")
        .when(col("delay") == 0, "No Delays")
        .when(col("delay") < 0, "Early Departure")
        .otherwise("Unknown").alias("delay_category")
    )
).show()


+-----+------+-----------+----------------+
|delay|origin|destination|  delay_category|
+-----+------+-----------+----------------+
|    6|   ABE|        ATL|Tolerable Delays|
|   -8|   ABE|        DTW| Early Departure|
|   -2|   ABE|        ATL| Early Departure|
|   -4|   ABE|        ATL| Early Departure|
|   -4|   ABE|        ATL| Early Departure|
|    0|   ABE|        ATL|       No Delays|
|   10|   ABE|        ATL|Tolerable Delays|
|   28|   ABE|        ATL|Tolerable Delays|
|   88|   ABE|        ATL|    Short Delays|
|    9|   ABE|        ATL|Tolerable Delays|
|   -6|   ABE|        ATL| Early Departure|
|   69|   ABE|        ATL|    Short Delays|
|    0|   ABE|        DTW|       No Delays|
|   -3|   ABE|        ATL| Early Departure|
|    0|   ABE|        DTW|       No Delays|
|    0|   ABE|        ATL|       No Delays|
|    0|   ABE|        DTW|       No Delays|
|    0|   ABE|        ATL|       No Delays|
|    0|   ABE|        ORD|       No Delays|
|    0|   ABE|        DTW|      

full fly info, joining with airport codes

In [19]:
airports = spark.read \
    .option("delimiter", "\t") \
    .option("header", True) \
    .csv("./data/7-airport-codes-na.txt")
    
airports.show(5, truncate=False)


+----------+-----+-------+----+
|City      |State|Country|IATA|
+----------+-----+-------+----+
|Abbotsford|BC   |Canada |YXX |
|Aberdeen  |SD   |USA    |ABR |
|Abilene   |TX   |USA    |ABI |
|Akron     |OH   |USA    |CAK |
|Alamosa   |CO   |USA    |ALS |
+----------+-----+-------+----+
only showing top 5 rows



In [20]:
df_join = (
    df_updated.alias("o")
        .join(airports.alias("ori"), (col("o.origin") == col("ori.IATA")), "left")
        .join(airports.alias("dest"), (col("o.destination") == col("dest.IATA")), "left")
    .selectExpr(
        "o.date",
        "o.delay",
        "o.distance",
        "o.origin",
        "o.destination",
        "ori.City as origin_city",
        "dest.City as destination_city",
        "ori.State as origin_state",
        "dest.State as destination_state",
        "ori.Country as origin_country",
        "dest.Country as destination_country"
    )
)

df_join.show(5)


+-----------+-----+--------+------+-----------+-----------+----------------+------------+-----------------+--------------+-------------------+
|       date|delay|distance|origin|destination|origin_city|destination_city|origin_state|destination_state|origin_country|destination_country|
+-----------+-----+--------+------+-----------+-----------+----------------+------------+-----------------+--------------+-------------------+
|01-01 12:45|    6|     602|   ABE|        ATL|  Allentown|         Atlanta|          PA|               GA|           USA|                USA|
|01-02 06:00|   -8|     369|   ABE|        DTW|  Allentown|         Detroit|          PA|               MI|           USA|                USA|
|01-02 12:45|   -2|     602|   ABE|        ATL|  Allentown|         Atlanta|          PA|               GA|           USA|                USA|
|01-02 06:05|   -4|     602|   ABE|        ATL|  Allentown|         Atlanta|          PA|               GA|           USA|                USA|